# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [62]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path

# Import API key
from api_keys import geoapify_key

In [63]:
hvplot

<module 'hvplot' from 'C:\\Users\\steve\\anaconda3\\Lib\\site-packages\\hvplot\\__init__.py'>

In [65]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,touros,-5.1989,-35.4608,29.40,66,34,5.07,BR,1693062700
1,1,labytnangi,66.6572,66.4183,6.43,83,96,4.27,RU,1693062701
2,2,port-aux-francais,-49.3500,70.2167,3.74,76,67,7.11,TF,1693062702
3,3,grytviken,-54.2811,-36.5092,-4.59,72,21,9.86,GS,1693062703
4,4,puerto natales,-51.7236,-72.4875,3.31,68,100,2.00,CL,1693062704


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [66]:
# Assuming your DataFrame contains the columns 'Lng', 'Lat', and 'Humidity'
coordinates_df = city_data_df[['Lng', 'Lat' , 'City', 'Humidity']]

# Create the map using hvplot.points
map_plot_1 = coordinates_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    color="City",
    title="City Humidity Map"
)

# Display the map
map_plot_1


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [67]:
# Narrow down cities that fit criteria (example: humidity > 70)
narrowed_down_df = city_data_df[city_data_df['Humidity'] > 70].copy()

# Drop any rows with null values
narrowed_down_df.dropna(inplace=True)

# Display sample data
narrowed_down_df.head()



,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,1,labytnangi,66.6572,66.4183,6.43,83,96,4.27,RU,1693062701
2,2,port-aux-francais,-49.3500,70.2167,3.74,76,67,7.11,TF,1693062702
3,3,grytviken,-54.2811,-36.5092,-4.59,72,21,9.86,GS,1693062703
5,5,sarnia,42.9787,-82.4041,20.07,84,100,1.54,CA,1693062652
6,6,kruisfontein,-34.0033,24.7314,11.60,73,73,9.63,ZA,1693062705


In [68]:
len(narrowed_down_df)

328

### Step 3: Create a new DataFrame called `hotel_df`.

In [69]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = narrowed_down_df.copy()[['City', 'Country', 'Lat', 'Lng', 'Humidity']]

# Add an empty column, "Hotel Name," to the DataFrame
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head(10)


,City,Country,Lat,Lng,Humidity,Hotel Name
1,labytnangi,RU,66.6572,66.4183,83,
2,port-aux-francais,TF,-49.3500,70.2167,76,
3,grytviken,GS,-54.2811,-36.5092,72,
5,sarnia,CA,42.9787,-82.4041,84,
6,kruisfontein,ZA,-34.0033,24.7314,73,
8,vernon,CA,50.2581,-119.2691,71,
9,st. john's,CA,47.5649,-52.7093,79,
10,dunedin,NZ,-45.8742,170.5036,72,
11,waitangi,NZ,-43.9535,-176.5597,81,
12,galveston,US,29.3669,-94.9669,78,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [56]:
hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]    
    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"
# Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
labytnangi - nearest hotel: Семь лиственниц
port-aux-francais - nearest hotel: Keravel
grytviken - nearest hotel: No hotel found
sarnia - nearest hotel: The Insignia Hotel, Sarnia, a Tribute Portfolio Hotel
kruisfontein - nearest hotel: No hotel found
vernon - nearest hotel: Okanagan Royal Park Inn
st. john's - nearest hotel: Alt Hotel
dunedin - nearest hotel: Scenic Hotel Dunedin City
waitangi - nearest hotel: Hotel Chathams
galveston - nearest hotel: Comfort Inn & Suites Texas City
papatowai - nearest hotel: No hotel found
port elizabeth - nearest hotel: Waterford Hotel
hithadhoo - nearest hotel: Pebbles Inn
yellowknife - nearest hotel: No hotel found
marsh harbour - nearest hotel: Lofty Fig Villas
port mathurin - nearest hotel: Escale Vacances
avarua - nearest hotel: Paradise Inn
puerto ayora - nearest hotel: Hostal La Mirada De Solitario George
hadibu - nearest hotel: No hotel found
stanley - nearest hotel: Hotel 52
uturoa - nearest hotel: Hawaiki Nui hotel
il

,City,Country,Lat,Lng,Humidity,Hotel Name,distance
1,labytnangi,RU,66.6572,66.4183,83,Семь лиственниц,NaN
2,port-aux-francais,TF,-49.3500,70.2167,76,Keravel,NaN
3,grytviken,GS,-54.2811,-36.5092,72,No hotel found,NaN
5,sarnia,CA,42.9787,-82.4041,84,"The Insignia Hotel, Sarnia, a Tribute Portfoli...",NaN
6,kruisfontein,ZA,-34.0033,24.7314,73,No hotel found,NaN
...,...,...,...,...,...,...,...
555,nemuro,JP,43.3236,145.5750,99,イーストハーバーホテル,NaN
556,bereznik,RU,62.8551,42.7072,71,Отель Прибрежный,NaN
557,beidao,CN,34.5686,105.8933,90,麦积大酒店,NaN
558,yingchuan,CN,34.1628,113.4639,98,No hotel found,NaN


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [70]:
# %%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE

In [76]:
%%capture --no-display
hotel_df = city_data_df[['Lng', 'Lat' , 'City']]

# Configure the map plot with OSM tile source
map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",  # Use OpenStreetMap (OSM) tile source
    frame_width=700,
    frame_height=500,
    color="City",
    title="City Hotel Map"
)

# # Display the map
map_plot



:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City)